# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [70]:
# Loading the environment
%load_ext dotenv
%dotenv 


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [71]:
import dask.dataframe as dd


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [72]:
import os
from glob import glob

# 1. Load the price data path from environment variables
price_data = os.getenv("PRICE_DATA")
assert price_data, "PRICE_DATA not set — check your .env file"

# 2. Build the recursive glob pattern
pattern = os.path.join(price_data, "**", "*.parquet")

# 3. Get all matching parquet file paths
parquet_files = glob(pattern, recursive=True)

# 4. Inspect (optional)
print(f"Looking in: {pattern}")
print(f"Found {len(parquet_files)} parquet files")
parquet_files[:5]


Looking in: ../../05_src/data/prices/**\*.parquet
Found 2943 parquet files


['../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.0.parquet']

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [77]:


# 1. Read parquet data
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

# 2. Create lagged columns and features
dd_shift = dd_px.groupby("ticker", group_keys=False).apply(
    lambda df: df.assign(
        Close_lag_1=df["Close"].shift(1),
        Adj_Close_lag_1=df["Adj Close"].shift(1)
    ),
    meta={
        "Date": "datetime64[ns]",
        "Open": "float64",
        "High": "float64",
        "Low": "float64",
        "Close": "float64",
        "Adj Close": "float64",
        "Volume": "float64",
        "source": "object",
        "Year": "int64",
        "Close_lag_1": "float64",
        "Adj_Close_lag_1": "float64"
    }
)

# 3. Add returns and high-low range
dd_feat = dd_shift.assign(
    Returns=lambda x: x["Close"] / x["Close_lag_1"] - 1,
    hi_lo_range=lambda x: x["High"] - x["Low"]
)

dd_feat.to_parquet(output_path, write_index=False)



+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [78]:
# Convert Dask to Pandas
pd_feat = dd_feat.compute()

# Add 10-day rolling average of Returns for each ticker
pd_feat["rolling_returns_10"] = pd_feat.groupby("ticker")["Returns"].transform(lambda x: x.rolling(10).mean())



Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

In [ ]:
# No, it was not strictly necessary. Dask supports rolling window operations using map_partitions or rolling, but it requires careful partitioning and index alignment.

# If the dataset is large and doesn't fit in memory, it's better to use Dask to avoid memory overflow. 

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.